## Import Data

In [1]:
#list the data
!ls Data

averages.csv
fivethirtyeight_partisan_lean_DISTRICTS.csv
fivethirtyeight_partisan_lean_STATES.csv
vote_predictions.csv


In [2]:
!ls

Capstone Project Proposal - 2.pdf  Data Wrangling.ipynb    README.md
Data				   PROPUBLICA_API_KEY.TXT


### Get API working

In [3]:
#Load ProPublica API key without exposing it to github. Request one yourself. They're free.
APIfile = open('./PROPUBLICA_API_KEY.TXT',)
APIkey = APIfile.read().replace('\n', '')
print(type(APIkey))
#print(APIkey)

<class 'str'>


In [4]:
#API request function
import requests

#create API key
headers = {'X-API-Key': APIkey}

#load status codes from API documentation(https://projects.propublica.org/api-docs/congress-api/#errors) into dictionary
status_codes = {
    200:'Successful Connection!',
    400:'Bad Request – Your request is improperly formed',
    403:'Forbidden – Your request did not include an authorization header',
    404:'Not Found – The specified record(s) could not be found',
    406:'Not Acceptable – You requested a format that isn’t json or xml',
    500:'Internal Server Error – We had a problem with our server. Try again later',
    503:'Service Unavailable – The service is currently not working. Please try again later'
    }

#load base url and test endpoint
url = 'https://api.propublica.org/congress/v1'
test_endpoint = '/members/new.json'

def API_Request(end_point, verbose=True):
#send a get request to restful API, print status and return response as a dictionary
    response = requests.get(url+end_point, headers=headers)
    #print the response status
    if verbose == True:
        try:
            print(status_codes[response.status_code])
        except: 
            print('API Failure, unknown status code: '+response.status_code)
 
    return response.json()

#test that the API is working. 
new_members = API_Request(test_endpoint)
print(new_members.keys()) # When the API works this should give the list of attributes per member
print('status:', new_members['status'])
print('no. results:', new_members['results'][0]['num_results'])

Successful Connection!
dict_keys(['status', 'copyright', 'results'])
status: OK
no. results: 20


In [5]:
#print what type of data we can get for each new member
new_members_data = new_members['results'][0]['members']
print(new_members_data[0].keys())


dict_keys(['id', 'api_uri', 'first_name', 'middle_name', 'last_name', 'suffix', 'party', 'chamber', 'state', 'start_date'])


In [6]:
#list some data for 20 members:
for member in new_members_data:
    print(member['first_name'], member['last_name']+':', member['party'])

Rick Scott: R
Michael San Nicolas: D
Jared Golden: D
Martha McSally: R
Mitt Romney: R
Joshua Hawley: R
Mike Braun: R
Carol Miller: R
Bryan Steil: R
Kim Schrier: D
Jennifer Wexton: D
Abigail Spanberger: D
Ben Cline: R
Denver Riggleman: R
Elaine Luria: D
Ben McAdams: D
Colin Allred: D
Sylvia Garcia: D
Chip Roy: R
Veronica Escobar: D


### The API is working.  Let's import voting data

In [7]:
#import test rollcal vote for the senate
recent_endpoint = '/senate/votes/recent.json'
senate_vote_recent = API_Request(recent_endpoint)
print('no. results', senate_vote_recent['results']['num_results'])

Successful Connection!
no. results 20


In [8]:
last_20_votes = senate_vote_recent['results']['votes']
print(last_20_votes[0].keys())

dict_keys(['congress', 'chamber', 'session', 'roll_call', 'source', 'url', 'vote_uri', 'bill', 'amendment', 'nomination', 'question', 'question_text', 'description', 'vote_type', 'date', 'time', 'result', 'tie_breaker', 'tie_breaker_vote', 'document_number', 'document_title', 'democratic', 'republican', 'independent', 'total'])


In [9]:
for vote in last_20_votes:
    print(vote['chamber'], vote['congress'], str(vote['roll_call'])+':', vote['description']+'\n')
    

Senate 116 105: Michael H. Park, of New York, to be United States Circuit Judge for the Second Circuit

Senate 116 104: Janet Dhillon, of Pennsylvania, to be a Member of the Equal Employment Opportunity Commission for a term expiring July 1, 2022

Senate 116 103: Janet Dhillon, of Pennsylvania, to be a Member of the Equal Employment Opportunity Commission for a term expiring July 1, 2022

Senate 116 102: Judith DelZoppo Pryor, of Ohio, to be a Member of the Board of Directors of the Export-Import Bank of the United States for a term expiring January 20, 2021

Senate 116 101: Spencer Bachus III, of Alabama, to be a Member of the Board of Directors of the Export-Import Bank of the United States for a term expiring January 20, 2023

Senate 116 100: Kimberly A. Reed, of West Virginia, to be President of the Export-Import Bank of the United States for a term expiring January 20, 2021

Senate 116 99: Joseph F. Bianco, of New York, to be United States Circuit Judge for the Second Circuit

Sen

In [10]:
import pandas as pd
last_20df = pd.DataFrame(last_20_votes)
last_20df.loc[:5,'bill':'description']

,bill,chamber,congress,date,democratic,description
0,{},Senate,116,2019-05-08,"{'yes': 0, 'no': 41, 'present': 0, 'not_voting...","Michael H. Park, of New York, to be United Sta..."
1,{},Senate,116,2019-05-08,"{'yes': 0, 'no': 41, 'present': 0, 'not_voting...","Janet Dhillon, of Pennsylvania, to be a Member..."
2,{},Senate,116,2019-05-08,"{'yes': 0, 'no': 42, 'present': 0, 'not_voting...","Janet Dhillon, of Pennsylvania, to be a Member..."
3,{},Senate,116,2019-05-08,"{'yes': 42, 'no': 0, 'present': 0, 'not_voting...","Judith DelZoppo Pryor, of Ohio, to be a Member..."
4,{},Senate,116,2019-05-08,"{'yes': 36, 'no': 5, 'present': 0, 'not_voting...","Spencer Bachus III, of Alabama, to be a Member..."
5,{},Senate,116,2019-05-08,"{'yes': 42, 'no': 0, 'present': 0, 'not_voting...","Kimberly A. Reed, of West Virginia, to be Pres..."


In [11]:
#use above to test get voting record for the latest bill
congress = last_20df.loc[0,'congress']
chamber = last_20df.loc[0,'chamber']
session = last_20df.loc[0,'session']
roll_call = last_20df.loc[0,'roll_call']

lv_endpoint = '/'+str(congress)+'/'+chamber.lower()+'/sessions/'+str(session)+'/votes/'+str(roll_call)+'.json'
print('using endpoint:', lv_endpoint)

latest_vote = API_Request(lv_endpoint)

using endpoint: /116/senate/sessions/1/votes/105.json
Successful Connection!


In [12]:
vote = latest_vote['results']['votes']['vote']
vote.keys()

dict_keys(['congress', 'session', 'chamber', 'roll_call', 'source', 'url', 'bill', 'amendment', 'nomination', 'question', 'question_text', 'description', 'vote_type', 'date', 'time', 'result', 'tie_breaker', 'tie_breaker_vote', 'document_number', 'document_title', 'democratic', 'republican', 'independent', 'total', 'positions'])

In [13]:
#load a dataframe of everyone's position on the latest vote:
latest_vote_positions_df = pd.DataFrame(vote['positions'])
latest_vote_positions_df

,dw_nominate,member_id,name,party,state,vote_position
0,0.323,A000360,Lamar Alexander,R,TN,Yes
1,-0.521,B001230,Tammy Baldwin,D,WI,No
2,0.534,B001261,John Barrasso,R,WY,Yes
3,-0.208,B001267,Michael Bennet,D,CO,Not Voting
4,0.614,B001243,Marsha Blackburn,R,TN,Yes
5,-0.411,B001277,Richard Blumenthal,D,CT,No
6,0.429,B000575,Roy Blunt,R,MO,Yes
7,-0.618,B001288,Cory Booker,D,NJ,Not Voting
8,0.401,B001236,John Boozman,R,AR,Yes
9,NaN,B001310,Mike Braun,R,IN,Yes


In [69]:
#Yeah, that's right! That's vote data!

#let's create a function that will import vote positions for a given rollcall vote.
def import_positions(congress, chamber, session, roll_call, verbose=False):
    '''imports all member positions of a given roll_call vote
        takes congress number 102-116 for House, 80-116 for Senate
        chamber(house|senate) session 1 for odd 2 for even-numbered years
        returns a dicitonary of vote positions and API status'''
    #construct endpoint for API request
    call_endpoint = '/'+str(congress)+'/'+chamber.lower()+'/sessions/'+str(session)+'/votes/'+str(roll_call)+'.json'
    
    #send endpoint to API function for request
    call_response = API_Request(call_endpoint, verbose)
    
    #extract the status of teh API request
    status = call_response['status']
    
    #try to extract and return the vote positions
    try:
        vote = call_response['results']['votes']['vote']
        positions = vote['positions']
        return positions, status
    except:
        return {}, status
    
    
#let's create a function that will import a given congressional session
def import_session(congress, chamber, session):
    '''import a congressional session worth of votes using the pro-publica API
        takes the congress number chamber and session as input and returns a data frame'''
    
    roll_call = 0
    status = 'OK'
    #if chamber == 'house': #handle the first rollcall and speaker vote
    #    roll_call += 2
    print('Importing ',congress,chamber,session)
    #positions, status = import_positions(congress,chamber,session,roll_call,verbose=True)
    #df = pd.DataFrame(positions).loc[:,'member_id':'vote_position']
    while status == 'OK':
        roll_call += 1
        positions, status = import_positions(congress,chamber,session,roll_call)
        call_df = pd.DataFrame(positions)
        dflng = len(call_df)
        if dflng > 0 and dflng < 440:
            try:
                df = pd.merge(df, call_df.loc[:,'member_id':'vote_position'], on=['member_id','name','party','state'], how='outer', suffixes=('','_'+str(roll_call))) 
                print('.', end='')
            except:
                #intialize df
                print('Initalizing df', roll_call)
                print('API status:', status)
                df = call_df
        else:
            print('(Size) skip vote', roll_call)
                
    return df

#test (remove when working)
#print('All Senate Votes for the current session:')
#latest_senate_1_df = import_session(116, 'senate', 1)
#latest_senate_1_df

In [70]:
#import vote poisiotns for 115th house, session 1
house_115_1_df = import_session(115, 'house', 1)
house_115_1_df

Importing  115 house 1
(Size) skip vote 1
Initalizing df 2
API status: OK
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................(Size) skip vote 711


,district,dw_nominate,member_id,name,party,state,vote_position,vote_position_3,vote_position_4,vote_position_5,...,vote_position_701,vote_position_702,vote_position_703,vote_position_704,vote_position_705,vote_position_706,vote_position_707,vote_position_708,vote_position_709,vote_position_710
0,5,0.508,A000374,Ralph Abraham,R,LA,Ryan (WI),Yes,Yes,No,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
1,12,-0.469,A000370,Alma Adams,D,NC,Pelosi,Not Voting,No,Yes,...,Yes,No,No,No,No,Yes,No,No,Yes,Yes
2,4,0.361,A000055,Robert B. Aderholt,R,AL,Ryan (WI),Yes,Yes,No,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
3,31,-0.285,A000371,Pete Aguilar,D,CA,Pelosi,No,No,Yes,...,Yes,No,No,No,No,Yes,No,No,Yes,Yes
4,12,0.611,A000372,Rick Allen,R,GA,Ryan (WI),Yes,Yes,No,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
5,3,0.648,A000367,Justin Amash,R,MI,Ryan (WI),Yes,Yes,No,...,No,No,No,Yes,No,Yes,No,No,No,Yes
6,2,0.375,A000369,Mark Amodei,R,NV,Ryan (WI),Yes,Yes,No,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
7,19,0.549,A000375,Jodey Arrington,R,TX,Ryan (WI),Yes,Yes,No,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Not Voting
8,36,0.693,B001291,Brian Babin,R,TX,Ryan (WI),Yes,Yes,No,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
9,2,0.432,B001298,Don Bacon,R,NE,Ryan (WI),Yes,Yes,No,...,No,Yes,Yes,Yes,Yes,Yes,Not Voting,Yes,Yes,Yes


In [30]:
#import vote postitions for 115th senate, session 1
senate_115_1_df = import_session(115, 'senate', 1)
senate_115_1_df

Importing  115 senate 1
Successful Connection!
....................................................................................................................................................................................................................................................................................................................................
Finished on roll call: 325
API status: ERROR


,member_id,name,party,state,vote_position,vote_position_2,vote_position_3,vote_position_4,vote_position_5,vote_position_6,...,vote_position_316,vote_position_317,vote_position_318,vote_position_319,vote_position_320,vote_position_321,vote_position_322,vote_position_323,vote_position_324,vote_position_325
0,A000360,Lamar Alexander,R,TN,Yes,No,No,No,No,No,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
1,B001230,Tammy Baldwin,D,WI,No,Yes,No,Yes,No,Yes,...,No,No,Not Voting,Not Voting,Yes,No,No,No,Yes,No
2,B001261,John Barrasso,R,WY,Yes,No,No,No,No,No,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
3,B001267,Michael Bennet,D,CO,No,Yes,No,Yes,No,Yes,...,No,No,Yes,Yes,Yes,No,No,No,Yes,No
4,B001277,Richard Blumenthal,D,CT,No,Yes,No,Yes,No,Yes,...,No,No,No,Yes,Yes,No,No,No,Yes,No
5,B000575,Roy Blunt,R,MO,Yes,No,Not Voting,Not Voting,No,No,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
6,B001288,Cory Booker,D,NJ,No,Yes,No,Yes,No,Yes,...,No,No,No,No,Yes,No,No,No,Yes,No
7,B001236,John Boozman,R,AR,Yes,No,No,No,No,No,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
8,B000944,Sherrod Brown,D,OH,No,Yes,No,Yes,No,Yes,...,No,No,No,No,Yes,No,No,No,Yes,No
9,B001135,Richard M. Burr,R,NC,Yes,No,No,No,Yes,No,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes


In [41]:
positions, status = import_positions(115,'house',1,3)
pd.DataFrame(positions)

,district,dw_nominate,member_id,name,party,state,vote_position
0,5,0.508,A000374,Ralph Abraham,R,LA,Yes
1,12,-0.469,A000370,Alma Adams,D,NC,Not Voting
2,4,0.361,A000055,Robert B. Aderholt,R,AL,Yes
3,31,-0.285,A000371,Pete Aguilar,D,CA,No
4,12,0.611,A000372,Rick Allen,R,GA,Yes
5,3,0.648,A000367,Justin Amash,R,MI,Yes
6,2,0.375,A000369,Mark Amodei,R,NV,Yes
7,19,0.549,A000375,Jodey Arrington,R,TX,Yes
8,36,0.693,B001291,Brian Babin,R,TX,Yes
9,2,0.432,B001298,Don Bacon,R,NE,Yes


In [36]:
test = pd.DataFrame(positions)
test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1292 entries, 0 to 1291
Data columns (total 7 columns):
district         1292 non-null object
dw_nominate      1292 non-null float64
member_id        1292 non-null object
name             1292 non-null object
party            1292 non-null object
state            1292 non-null object
vote_position    1292 non-null object
dtypes: float64(1), object(6)
memory usage: 70.7+ KB


In [37]:
names = test['name'].tolist()
nameset = set(names)
len(nameset)

436

In [39]:
ids = test['member_id'].tolist()
idset = set(ids)
len(idset)

436